This notebook reproduces creation of CondBERT vocabulary. 

In [1]:
!git clone https://github.com/s-nlp/detox

Cloning into 'detox'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 180 (delta 50), reused 118 (delta 14), pack-reused 0
Receiving objects: 100% (180/180), 17.92 MiB | 12.89 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [2]:
! pip install -r detox/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.8/400.8 KB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 66.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

# 0. Prerequisites

In [3]:
import os
import sys

def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

In [4]:
add_sys_path('detox/emnlp2021/style_transfer/condBERT')

/content/detox/emnlp2021/style_transfer/condBERT


In [5]:
from importlib import reload
import condbert
reload(condbert)
from condbert import CondBertRewriter

# 1. Loading BERT

In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.1 MB/s eta 0:00:00
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.4 MB/s eta 0:00:00


In [7]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
from tqdm.auto import tqdm, trange
import pandas as pd

In [131]:
device = torch.device('cuda:0')
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.to(device);

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# 2. Preparing the vocabularires.


- old-words.txt
- modern-words.txt
- word2coef.pkl
- token_shakespearean.txt

These files should be prepared once. 

In [98]:
dataset = pd.read_csv('shakespearen.csv')

Используемый датасет параллельный, поэтому мы возьмем разные части датасета для разных стилей.

In [17]:
import re

In [25]:
def preprocess_text(text):
  for a in '.,?!:;-)(':
    text = text.replace(a, ' '+a)
  text = text.replace('"', '')
  text = text.replace("'", "")
  text = re.sub(' +', ' ', text)
  text = text.lower()
  return text

In [28]:
dataset['og'] = dataset['og'].apply(preprocess_text)
dataset['t'] = dataset['t'].apply(preprocess_text)

In [29]:
dataset

,Unnamed: 0,id,og,t
0,0,42928-1500614319216-63344,you do not meet a man but frowns :,every man you meet these days is frowning .
1,1,42928-1500614326583-89821,our bloods no more obey the heavens than our ...,our bodies are in agreement with the planetar...
2,2,A-63849,but whats the matter ?,whats wrong ?
3,3,42930-1500614347266-80123,"his daughter , and the heir ofs kingdom , who...","the king wanted his daughter , the only heir t..."
4,4,42930-1500614355280-38326,shes wedded ; her husband banishd ; she impri...,"shes married , her husband is banished , shes..."
...,...,...,...,...
51782,205,40471-1502742711264-62106,he hath not told us of the captain yet .,he hasnt told us about that captain yet .
51783,206,40471-1502742725153-49869,"when that is known and golden time convents ,...",when thats taken care of and the time is conve...
51784,207,40471-1502742735040-34786,"meantime , sweet sister , we will not part fr...","until then , sweet sister -in -law , we wont l..."
51785,208,40471-1502742744235-12820,"cesario , come , for so you shall be , while ...","cesario , come here . youll be cesario to me w..."


In [32]:
old_corpus_path = dataset['og'][:20000].to_list()
modern_corpus_path = dataset['t'][20000:40000].to_list()

### 2.1 Preparing the DRG-like vocabularies

In [33]:
import os
import argparse
import numpy as np
from tqdm import tqdm
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



class NgramSalienceCalculator():
    def __init__(self, old_corpus, modern_corpus, use_ngrams=False):
        ngrams = (1, 3) if use_ngrams else (1, 1)
        self.vectorizer = CountVectorizer(ngram_range=ngrams)

        old_count_matrix = self.vectorizer.fit_transform(old_corpus)
        self.old_vocab = self.vectorizer.vocabulary_
        self.old_counts = np.sum(old_count_matrix, axis=0)

        modern_count_matrix = self.vectorizer.fit_transform(modern_corpus)
        self.modern_vocab = self.vectorizer.vocabulary_
        self.modern_counts = np.sum(modern_count_matrix, axis=0)

    def salience(self, feature, attribute='old', lmbda=0.5):
        assert attribute in ['old', 'modern']
        if feature not in self.old_vocab:
            old_count = 0.0
        else:
            old_count = self.old_counts[0, self.old_vocab[feature]]

        if feature not in self.modern_vocab:
            modern_count = 0.0
        else:
            modern_count = self.modern_counts[0, self.modern_vocab[feature]]

        if attribute == 'old':
            return (old_count + lmbda) / (modern_count + lmbda)
        else:
            return (modern_count + lmbda) / (old_count + lmbda)


In [34]:
from collections import Counter
c = Counter()

for corpus in [old_corpus_path, modern_corpus_path]:
  for sent in corpus:
    for tok in sent.strip().split():
      c[tok] += 1

print(len(c))

28434


In [35]:
vocab = {w for w, _ in c.most_common() if _ > 0}  # if we took words with > 1 occurences, vocabulary would be x2 smaller, but we'll survive this size
print(len(vocab))

28434


In [37]:
corpus_old = [' '.join([w if w in vocab else '<unk>' for w in sent.strip().split()]) for sent in old_corpus_path]
corpus_modern = [' '.join([w if w in vocab else '<unk>' for w in sent.strip().split()]) for sent in modern_corpus_path]

In [38]:
old_out_name = 'old-words.txt'
modern_out_name = 'modern-words.txt'

In [39]:
threshold = 4

In [40]:
sc = NgramSalienceCalculator(corpus_old, corpus_modern, False)
seen_grams = set()

with open(old_out_name, 'w') as old_out, open(modern_out_name, 'w') as modern_out:
    for gram in set(sc.old_vocab.keys()).union(set(sc.modern_vocab.keys())):
        if gram not in seen_grams:
            seen_grams.add(gram)
            old_salience = sc.salience(gram, attribute='old')
            modern_salience = sc.salience(gram, attribute='modern')
            if old_salience > threshold:
                old_out.writelines(f'{gram}\n')
            elif modern_salience > threshold:
                modern_out.writelines(f'{gram}\n')

## 2.2 Evaluating word shakespearities with a logistic regression

In [41]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

In [42]:
X_train = corpus_old + corpus_modern
y_train = [1] * len(corpus_old) + [0] * len(corpus_modern)
pipe.fit(X_train, y_train);

In [43]:
coefs = pipe[1].coef_[0]
coefs.shape

(23329,)

In [44]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [45]:
import pickle
with open('word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

## 2.3 Labelling BERT tokens by shakespearity

In [46]:
from collections import defaultdict
old_counter = defaultdict(lambda: 1)
modern_counter = defaultdict(lambda: 1)

for text in tqdm(corpus_old):
    for token in tokenizer.encode(text):
        old_counter[token] += 1
for text in tqdm(corpus_modern):
    for token in tokenizer.encode(text):
        modern_counter[token] += 1

100%|██████████| 20000/20000 [00:11<00:00, 1748.21it/s]


In [47]:
token_shakespearean = [old_counter[i] / (modern_counter[i] + old_counter[i]) for i in range(len(tokenizer.vocab))]

In [48]:
with open('token_shakespearean.txt', 'w') as f:
    for t in token_shakespearean:
        f.write(str(t))
        f.write('\n')

# 3. Setting up the model

### 3.1 Loading the vocabularies

In [49]:
with open('old-words.txt', 'r') as f:
    s = f.readlines()
old_words = list(map(lambda x: x[:-1], s))

with open('modern-words.txt', 'r') as f:
    s = f.readlines()
modern_words = list(map(lambda x: x[:-1], s))

In [50]:
import pickle
with open('word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [60]:
token_shakespearean = []
with open('token_shakespearean.txt', 'r') as f:
    for line in f.readlines():
        token_shakespearean.append(float(line))
token_shakespearean = np.array(token_shakespearean)
token_shakespearean = np.maximum(0, np.log(1/(1/token_shakespearean-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-', ';']:
    token_shakespearean[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_shakespearean[tokenizer.encode(tok)][1] = 0

## Applying the model

In [132]:
reload(condbert)
from condbert import CondBertRewriter

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=old_words,
    pos_words=modern_words,
    word2coef=word2coef,
    token_toxicities=token_shakespearean,
    predictor=None
)

In [133]:
print(editor.translate('Thou canst not fear us , Pompey , with thy sails .', prnt=False))

you cant not fear us , pttai , with your sails .


In [54]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)


In [134]:
from multiword import masked_token_predictor_bert
reload(masked_token_predictor_bert)
from multiword.masked_token_predictor_bert import MaskedTokenPredictorBert

In [135]:
predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0)
editor.predictor = predictor

def adjust_logits(logits, label):
    return logits - editor.token_toxicities * 3

predictor.logits_postprocessor = adjust_logits

print(editor.replacement_loop('Thou canst not fear us , Pompey , with thy sails .', verbose=False))

you can not fear us , po , with your sails .


In [61]:
%%time
print(editor.replacement_loop('Thou canst not fear us , Pompey , with thy sails .', verbose=False, n_units=1))

you can not fear us , po , with your sails .
CPU times: user 2.43 s, sys: 3.25 ms, total: 2.43 s
Wall time: 2.64 s


In [62]:
%time
print(editor.replacement_loop('Thou canst not fear us , Pompey , with thy sails .', verbose=False, n_units=3))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
you filth not fear us , po , with your sails .


In [63]:
%%time
print(editor.replacement_loop('Thou canst not fear us , Pompey , with thy sails .', verbose=False, n_units=10))

you filth not fear us , po , with your sails .
CPU times: user 8.98 s, sys: 0 ns, total: 8.98 s
Wall time: 11.3 s


In [ ]:
!pip install flair==0.11

In [65]:
import choosers
reload(choosers)
from choosers import EmbeddingSimilarityChooser

In [136]:
predictor = MaskedTokenPredictorBert(
    model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0, 
    confuse_bert_args=True, # this argument deteriorates quality but is used for backward compatibility
)
editor.predictor = predictor

def adjust_logits(logits, label=0):
    return logits - editor.token_toxicities * 10

predictor.logits_postprocessor = adjust_logits

cho = EmbeddingSimilarityChooser(sim_coef=100, tokenizer=tokenizer)

In [67]:
old_test_ds = dataset['og'][40000:].to_list()

In [156]:
for i, sent in enumerate(tqdm(old_test_ds[10:15])):
  inp = sent.strip()
  out = editor.replacement_loop(inp, verbose=False, chooser=cho, n_top=10, n_tokens=(1,2,3), n_units=1)
  print('\n', inp, '\n', out)

 20%|██        | 1/5 [00:00<00:02,  1.46it/s]


 i serve you , madam . your graces are right welcome . 
 i serve you , madam . your " " " are right welcome .


 40%|████      | 2/5 [00:02<00:04,  1.40s/it]


 good dawning to thee , friend . art of this house ? 
 good the to " " " , friend . " . of this house ?


 60%|██████    | 3/5 [00:03<00:02,  1.03s/it]


 ay . 
 . " . .


 80%|████████  | 4/5 [00:05<00:01,  1.46s/it]


 where may we set our horses ? 
 where they we they our horses ?


100%|██████████| 5/5 [00:06<00:00,  1.36s/it]


 i th mire . 
 i i " " " .


### Переведем с помощью conbert 1000 предложений для дальнейшей оценки качества и сохраним ее в файл model_outputs.txt

In [138]:
import random

In [152]:
test1000 = random.sample(old_test_ds, 1000)

In [157]:
model_outputs = []
for i, sent in enumerate(tqdm(test1000)):
  inp = sent.strip()
  out = editor.replacement_loop(inp, verbose=False, chooser=cho, n_top=10, n_tokens=(1,2,3), n_units=1)
  model_outputs.append(out)

100%|██████████| 1000/1000 [27:21<00:00,  1.64s/it]


In [160]:
with open('model_outputs.txt', 'w') as f:
    for sent in model_outputs:
      f.writelines(f'{sent}\n')